# Testing sims

In [1]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np

In [2]:
backtest_years = [2022]
pbp = nfl.import_pbp_data(backtest_years, downcast=True, cache=False, alt_path=None)

2022 done.
Downcasting floats.


In [3]:
game_info_cols = [
    'game_id', 'game_date', 'home_team', 'away_team', 'season_type', 'week', 'season',
    'posteam', 'defteam', 'posteam_type'
]

scoring_stats = [
    'total_home_score', 'total_away_score'
]

offensive_stats = [
    'yards_gained', 'pass_attempt', 'rush_attempt', 'pass_touchdown', 'rush_touchdown',
    'interception', 'fumble'
]

defensive_stats = [

]

special_teams_stats = [
    'punt_attempt', 'field_goal_result', 'field_goal_attempt'
]

cols = game_info_cols + scoring_stats + offensive_stats + defensive_stats + special_teams_stats

In [5]:
for col in pbp.columns:
    print(col)

play_id
game_id
old_game_id
home_team
away_team
season_type
week
posteam
posteam_type
defteam
side_of_field
yardline_100
game_date
quarter_seconds_remaining
half_seconds_remaining
game_seconds_remaining
game_half
quarter_end
drive
sp
qtr
down
goal_to_go
time
yrdln
ydstogo
ydsnet
desc
play_type
yards_gained
shotgun
no_huddle
qb_dropback
qb_kneel
qb_spike
qb_scramble
pass_length
pass_location
air_yards
yards_after_catch
run_location
run_gap
field_goal_result
kick_distance
extra_point_result
two_point_conv_result
home_timeouts_remaining
away_timeouts_remaining
timeout
timeout_team
td_team
td_player_name
td_player_id
posteam_timeouts_remaining
defteam_timeouts_remaining
total_home_score
total_away_score
posteam_score
defteam_score
score_differential
posteam_score_post
defteam_score_post
score_differential_post
no_score_prob
opp_fg_prob
opp_safety_prob
opp_td_prob
fg_prob
safety_prob
td_prob
extra_point_prob
two_point_conversion_prob
ep
epa
total_home_epa
total_away_epa
total_home_rush_epa


In [23]:
# Set pandas display options
pd.set_option('display.max_columns', None)
df = pbp.loc[~pbp['posteam_type'].isna(), :].copy()
df.fillna(0, inplace=True)

# Score columns
df['home_score'] = df['total_home_score'].astype(int)
df['away_score'] = df['total_away_score'].astype(int)

# Passing columns
df['posteam_type'] = df['posteam_type'].map({'home': 1, 'away': 0}).astype(int)
df['home_pass_yards'] = np.where((df['pass_attempt'].astype(int)) & (df['posteam_type']), df['yards_gained'], 0)
df['away_pass_yards'] = np.where((df['pass_attempt'].astype(int)) & (~df['posteam_type']), df['yards_gained'], 0)
df['home_pass_atts'] = np.where((df['pass_attempt'].astype(int)) & (df['posteam_type']), 1, 0)
df['away_pass_atts'] = np.where((df['pass_attempt'].astype(int)) & (~df['posteam_type']), 1, 0)
df['home_pass_tds'] = np.where((df['pass_touchdown'].astype(int)) & (df['posteam_type']), 1, 0)
df['away_pass_tds'] = np.where((df['pass_touchdown'].astype(int)) & (~df['posteam_type']), 1, 0)
df['home_pass_ints'] = np.where((df['interception'].astype(int)) & (df['posteam_type']), 1, 0)
df['away_pass_ints'] = np.where((df['interception'].astype(int)) & (~df['posteam_type']), 1, 0)

# Rushing columns
df['home_rush_yards'] = np.where((df['rush_attempt'].astype(int)) & (df['posteam_type']), df['yards_gained'], 0)
df['away_rush_yards'] = np.where((df['rush_attempt'].astype(int)) & (~df['posteam_type']), df['yards_gained'], 0)
df['home_rush_atts'] = np.where((df['rush_attempt'].astype(int)) & (df['posteam_type']), 1, 0)
df['away_rush_atts'] = np.where((df['rush_attempt'].astype(int)) & (~df['posteam_type']), 1, 0)
df['home_rush_tds'] = np.where((df['rush_touchdown'].astype(int)) & (df['posteam_type']), 1, 0)
df['away_rush_tds'] = np.where((df['rush_touchdown'].astype(int)) & (~df['posteam_type']), 1, 0)
df['home_rush_fums'] = np.where((df['fumble'].astype(int)) & (df['posteam_type']), 1, 0)
df['away_rush_fums'] = np.where((df['fumble'].astype(int)) & (~df['posteam_type']), 1, 0)

# Defensive columns
df['home_def_ints'] = np.where((df['interception'].astype(int)) & (~df['posteam_type']), 1, 0)
df['away_def_ints'] = np.where((df['interception'].astype(int)) & (df['posteam_type']), 1, 0)
df['home_def_fums'] = np.where((df['fumble'].astype(int)) & (~df['posteam_type']), 1, 0)
df['away_def_fums'] = np.where((df['fumble'].astype(int)) & (df['posteam_type']), 1, 0)
df['home_def_safeties'] = np.where((df['safety'].astype(int)) & (~df['posteam_type']), 1, 0)
df['away_def_safeties'] = np.where((df['safety'].astype(int)) & (df['posteam_type']), 1, 0)

# Special teams columns
df['home_kickoff_atts'] = np.where((df['kickoff_attempt'].astype(int)) & (df['posteam_type']), 1, 0)
df['away_kickoff_atts'] = np.where((df['kickoff_attempt'].astype(int)) & (~df['posteam_type']), 1, 0)
df['home_punt_atts'] = np.where((df['punt_attempt'].astype(int)) & (df['posteam_type']), 1, 0)
df['away_punt_atts'] = np.where((df['punt_attempt'].astype(int)) & (~df['posteam_type']), 1, 0)
df['home_fg_atts'] = np.where((df['field_goal_attempt'].astype(int)) & (df['posteam_type']), 1, 0)
df['away_fg_atts'] = np.where((df['field_goal_attempt'].astype(int)) & (~df['posteam_type']), 1, 0)
df['home_fg_made'] = np.where((df['field_goal_result'] == 'made') & (df['posteam_type']), 1, 0)
df['away_fg_made'] = np.where((df['field_goal_result'] == 'made') & (~df['posteam_type']), 1, 0)
df['home_fg_yards'] = np.where((df['field_goal_result'] == 'made') & (df['posteam_type']), df['kick_distance'], 0)
df['away_fg_yards'] = np.where((df['field_goal_result'] == 'made') & (~df['posteam_type']), df['kick_distance'], 0)

/tmp/ipykernel_27327/298149200.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['home_pass_yards'] = np.where((df['pass_attempt'].astype(int)) & (df['posteam_type']), df['yards_gained'], 0)
/tmp/ipykernel_27327/298149200.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['away_pass_yards'] = np.where((df['pass_attempt'].astype(int)) & (~df['posteam_type']), df['yards_gained'], 0)
/tmp/ipykernel_27327/298149200.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

In [26]:
game_cols = [
    'game_id', 'game_date', 'home_team', 'away_team', 'season_type', 'week', 'season',
]

df = (
    df
    .groupby(game_cols)
    .agg(
        {
            'home_score': 'max', 'away_score': 'max',
            'home_pass_yards': 'sum', 'away_pass_yards': 'sum',
            'home_pass_atts': 'sum', 'away_pass_atts': 'sum',
            'home_pass_tds': 'sum', 'away_pass_tds': 'sum',
            'home_pass_ints': 'sum', 'away_pass_ints': 'sum',
            'home_rush_yards': 'sum', 'away_rush_yards': 'sum',
            'home_rush_atts': 'sum', 'away_rush_atts': 'sum',
            'home_rush_tds': 'sum', 'away_rush_tds': 'sum',
            'home_rush_fums': 'sum', 'away_rush_fums': 'sum',
            'home_def_ints': 'sum', 'away_def_ints': 'sum',
            'home_def_fums': 'sum', 'away_def_fums': 'sum',
            'home_def_safeties': 'sum', 'away_def_safeties': 'sum',
            'home_kickoff_atts': 'sum', 'away_kickoff_atts': 'sum',
            'home_punt_atts': 'sum', 'away_punt_atts': 'sum',
            'home_fg_atts': 'sum', 'away_fg_atts': 'sum',
            'home_fg_made': 'sum', 'away_fg_made': 'sum',
            'home_fg_yards': 'sum', 'away_fg_yards': 'sum',
        }
    )
    .reset_index()
)
df['spread'] = df['home_score'] - df['away_score']
df['total'] = df['home_score'] + df['away_score']

In [27]:
df

,game_id,game_date,home_team,away_team,season_type,week,season,home_score,away_score,home_pass_yards,away_pass_yards,home_pass_atts,away_pass_atts,home_pass_tds,away_pass_tds,home_pass_ints,away_pass_ints,home_rush_yards,away_rush_yards,home_rush_atts,away_rush_atts,home_rush_tds,away_rush_tds,home_rush_fums,away_rush_fums,home_def_ints,away_def_ints,home_def_fums,away_def_fums,home_def_safeties,away_def_safeties,home_kickoff_atts,away_kickoff_atts,home_punt_atts,away_punt_atts,home_fg_atts,away_fg_atts,home_fg_made,away_fg_made,home_fg_yards,away_fg_yards,spread,total
0,2022_01_BAL_NYJ,2022-09-11,NYJ,BAL,REG,1,2022,9,24,297.0,211.0,62,32,1,3,1,1,83.0,63.0,17,21,0,0,2,1,1,1,1,2,0,0,5,3,6,6,2,1,1,1,45.0,24.0,-15,33
1,2022_01_BUF_LA,2022-09-08,LA,BUF,REG,1,2022,10,31,191.0,292.0,48,33,1,3,3,2,52.0,121.0,18,25,0,1,1,2,2,3,2,1,0,0,6,2,4,0,1,1,1,1,57.0,41.0,-21,41
2,2022_01_CLE_CAR,2022-09-11,CAR,CLE,REG,1,2022,24,26,207.0,138.0,31,35,1,1,1,0,54.0,217.0,19,39,2,1,6,0,0,1,0,6,0,0,7,5,5,4,1,4,1,4,34.0,154.0,-2,50
3,2022_01_DEN_SEA,2022-09-12,SEA,DEN,REG,1,2022,17,16,177.0,330.0,30,44,2,1,0,0,76.0,103.0,19,20,0,0,3,2,0,0,2,3,0,0,4,4,2,1,1,4,1,3,49.0,96.0,1,33
4,2022_01_GB_MIN,2022-09-11,MIN,GB,REG,1,2022,23,7,269.0,227.0,33,43,2,0,0,1,126.0,111.0,28,18,0,1,0,1,1,0,1,0,0,0,2,6,5,4,3,0,3,0,113.0,0.0,16,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,2022_20_JAX_KC,2023-01-21,KC,JAX,POST,20,2022,27,20,218.0,205.0,37,41,3,1,0,1,144.0,144.0,30,19,0,1,0,1,1,0,1,0,0,0,5,6,4,4,2,2,2,2,100.0,89.0,7,47
280,2022_20_NYG_PHI,2023-01-21,PHI,NYG,POST,20,2022,38,7,148.0,109.0,25,32,2,0,0,1,268.0,118.0,44,20,3,1,1,1,1,0,1,1,0,0,2,7,3,5,1,0,1,0,30.0,0.0,31,45
281,2022_21_CIN_KC,2023-01-29,KC,CIN,POST,21,2022,23,20,326.0,238.0,48,46,2,1,0,2,42.0,71.0,20,17,0,1,2,0,2,0,0,2,0,0,4,6,4,4,3,2,3,2,112.0,53.0,3,43
282,2022_21_SF_PHI,2023-01-29,PHI,SF,POST,21,2022,31,7,121.0,83.0,26,21,0,0,0,0,148.0,81.0,44,24,4,1,0,3,0,0,3,0,0,0,2,6,4,4,1,0,1,0,31.0,0.0,24,38


In [28]:
df.to_csv("/home/tylerengland/NFL/backend/test/data.csv")